In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('dictionary.csv')
df


,ID,Word,Alternative,Definition
0,0,a,kin,"PARTICLE (emphasis, emotion, or confirmation)"
1,1,akesi,NaN,"NOUN non-cute animal; reptile, amphibian"
2,2,ala,NaN,"ADJECTIVE no, not, zero"
3,3,alasa,NaN,"VERB to hunt, forage"
4,4,ale,ali,"ADJECTIVE all; abundant, countless, bountiful,..."
...,...,...,...,...
115,115,wan,NaN,"ADJECTIVE unique, united NUMBER one"
116,116,waso,NaN,"NOUN bird, flying creature, winged animal"
117,117,wawa,NaN,"ADJECTIVE strong, powerful; confident, sure; e..."
118,118,weka,NaN,"ADJECTIVE absent, away, ignored"


In [31]:
syllables = ['0', '1', '2']
space = ' '

In [32]:
# Define a function to convert to any base n
def convert_to_base_n(x, base):
    if x == 0:
        return '0'

    base_str = ''
    while x > 0:
        remainder = x % base
        base_str = str(remainder) + base_str
        x //= base
    
    return base_str

# Specify the base you want (e.g., base 3)
base_n = len(syllables)

# Add a new column with base-n indices
df[f'Base-{base_n} Index'] = df.index.map(lambda x: convert_to_base_n(x, base_n))

In [33]:
print(df.to_string())

      ID     Word  Alternative                                                                                                                        Definition Base-4 Index Base-5 Index Base-2 Index Base-3 Index
0      0        a          kin                                                                                     PARTICLE (emphasis, emotion, or confirmation)            0            0            0            0
1      1    akesi          NaN                                                                                          NOUN non-cute animal; reptile, amphibian            1            1            1            1
2      2      ala          NaN                                                                                                           ADJECTIVE no, not, zero            2            2           10            2
3      3    alasa          NaN                                                                                                              VERB to 

In [34]:
!pip3 install pydub

In [35]:
def word_to_index(word):
    lines = !cat dictionary.csv | grep \,{word}\,
    line = lines[0]
    index = line.split(",")[0]
    return index 

def word_to_base_n(word):
    return df.iloc[int(word_to_index(word))][f"Base-{len(syllables)} Index"]

word_to_base_n('pona')

'10010'

In [36]:
text_src = "ken la ona li lon tomo sona."
text = text_src.replace(",", " ")
text = text.replace(":", " ")
text = text.replace(".", "  ")
text_base_n = ' '.join([(word_to_base_n(word) if word else word) for word in text.split(" ")])
text_base_n

'220 1020 2121 1110 1122 11001 10121  '

In [37]:
import numpy as np
from scipy.io.wavfile import write

# Define a function to generate a sine wave for a given frequency and duration
def generate_sine_wave(frequency, duration_ms, amplitude=1.0, sample_rate=44100):
    t = np.linspace(0, duration_ms / 1000, int(sample_rate * (duration_ms / 1000)), endpoint=False)
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return wave

# Define a function to generate silence for a given duration
def generate_silence(duration_ms, sample_rate=44100):
    return np.zeros(int(duration_ms * sample_rate / 1000))

# Define the number of notes in an octave and their frequencies (adjust as needed)
num_notes = 12
base_frequency = 440.0  # A4, the 4th A on a piano
octave = [base_frequency * 2**(i/12) for i in range(num_notes)]

# Define the duration of each note in milliseconds (adjust as needed)
note_duration_ms = 500

# Define the sequence of note indices (adjust as needed)
note_sequence = ['1', '2', '3', '1', '2', ' ', '2']

# Generate and concatenate audio for the note sequence
def get_sequence_wave(note_sequence, note_duration_ms=note_duration_ms):
    sequence_wave = np.concatenate([
        generate_sine_wave(octave[int(note_index)], note_duration_ms) if note_index.isdecimal() else generate_silence(note_duration_ms)
        for note_index in note_sequence
    ])
    return sequence_wave

# Save the concatenated audio as a WAV file
write('sequence.wav', 44100, get_sequence_wave(note_sequence).astype(np.float32))


In [38]:
print(text, text_base_n)
wave = get_sequence_wave([*text_base_n], note_duration_ms=100).astype(np.float32)
write('sequence.wav', 44100, wave)
import IPython
IPython.display.Audio("sequence.wav")

ken la ona li lon tomo sona   220 1020 2121 1110 1122 11001 10121  
